<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/076_Clean_Tool_Dependency_Injection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## ✅ What You Should Be Learning in This Lecture

### 🧠 **Main Concept: Clean Tool Design with Dependency Injection**

This lecture is teaching you how to **build agent tools that are clean, testable, decoupled, and reusable** — by using `ActionContext` to inject dependencies rather than hardcoding them.

---

## 📌 Key Ideas to Focus On

### 1. **Why Dependency Injection Matters**

* Without DI, tools would need to know *how to find their resources* (e.g., memory, LLMs, APIs).
* This leads to **tight coupling**, which means poor reusability and poor testability.
* With DI via `ActionContext`, the tools get exactly what they need **in a clean, declarative way**.

---

### 2. **The `ActionContext` Class**

You’ve seen this already, but it’s now formalized as a critical design component:

```python
class ActionContext:
    def __init__(self, properties: Dict = None):
        self.context_id = str(uuid.uuid4())
        self.properties = properties or {}

    def get(self, key: str, default=None):
        return self.properties.get(key, default)

    def get_memory(self):
        return self.properties.get("memory", None)
```

Focus on how this lets you:

* Provide tools with memory, LLM, and any service like `auth_token` or `SMTP`.
* Dynamically swap out these resources at runtime (e.g., for testing, production, role-switching).

---

### 3. **Refactored Tool with Dependency Injection**

Before:

```python
def analyze_code_quality(code: str) -> str:
    # Can't access memory without tight coupling
```

After:

```python
def analyze_code_quality(action_context: ActionContext, code: str) -> str:
    memory = action_context.get_memory()
    # Now you can review history + use LLM injected into action_context
```

✅ **Why this is important:**
This version is **agent-agnostic**. It doesn’t care what kind of agent is running it. It just expects the required context to be present — that's flexibility by design.

---

## 🧪 Practical Insight

This entire pattern:

* Makes **unit testing tools** easier (you can inject fake dependencies).
* Makes it easier to **swap tools in and out** in an agent ecosystem.
* Prevents tools from having hidden, implicit dependencies — everything is passed in and transparent.





## 🧰 Clean Tool Dependency Injection with the Environment

Now that we have our `ActionContext` to hold shared resources and dependencies, we face a new challenge: **how do we get it to just the tools that need it?** We need a solution that provides dependencies **selectively**, giving each tool access to **only the resources it requires**.

Consider that many tools are simple and self-contained, needing only their **explicit parameters**. A basic string manipulation tool shouldn’t receive memory access or authentication tokens it doesn’t use. Not only would this add unnecessary complexity, but it could also create **security concerns** by exposing sensitive information to tools that don’t need it.

---

### 🧪 Example: A Simple Tool with No Dependencies

```python
@register_tool(description="Convert text to uppercase")
def to_uppercase(text: str) -> str:
    """Convert input text to uppercase."""
    return text.upper()
```

---

### 🔐 Example: A Tool that Requires Authentication

```python
@register_tool(description="Update user profile")
def update_profile(action_context: ActionContext,
                  username: str,
                  _auth_token: str) -> dict:
    """Update a user's profile information."""
    # This tool needs auth_token from context
    return make_authenticated_request(_auth_token, username)
```






## 🧠 Avoiding Naive Dependency Injection

The naive solution would be to modify our agent to pass the `ActionContext` to **every tool**. However, this would not only **clutter our agent’s orchestration logic** with dependency management details but also force **unnecessary dependencies** on tools that don’t need them.

Every time the agent calls a tool, it would need to:

* ✅ Check if the tool needs the `ActionContext`
* ➕ Add it to the arguments if needed
* 🔍 Check for any other special dependencies the tool requires
* 🧱 Ensure these don’t conflict with the actual parameters the tool expects

This quickly becomes messy. Here’s what it might look like:

```python
def handle_agent_response(self, action_context: ActionContext, response: str) -> dict:
    """Handle action with dependency injection in the agent."""
    action_def, action = self.get_action(response)
    
    # Agent has to manage all this dependency logic
    args = action["args"].copy()
    if needs_action_context(action_def):
        args["action_context"] = action_context
    if needs_auth_token(action_def):
        args["_auth_token"] = action_context.get("auth_token")
    if needs_user_config(action_def):
        args["_user_config"] = action_context.get("user_config")
        
    result = action_def.execute(**args)
    return result
```

---

This is exactly the kind of complexity we want to **keep out of our agent**.

* 🧠 The agent should **focus on deciding what actions to take**,
* 🔒 Not on managing how dependencies get passed to tools.
* 🔧 This approach also complicates **security** and **separation of concerns**,
  as **every tool would potentially have access to all dependencies**.

---

## ✅ A Cleaner Approach: Let the Environment Handle It

Instead, we can implement this logic in our **environment system**, which can:

* Examine each tool’s requirements
* Provide only the dependencies it specifically requests

Here’s how much cleaner the agent’s code becomes:

```python
def handle_agent_response(self, action_context: ActionContext, response: str) -> dict:
    """Handle action without dependency management."""
    action_def, action = self.get_action(response)
    result = self.environment.execute_action(self, action_context, action_def, action["args"])
    return result
```

The agent simply **passes everything to the environment**, and lets it handle the details.
The environment can then analyze each tool’s signature and provide **exactly** the dependencies it needs – **no more, no less**.





## 🔧 Updating the Environment to Provide Dependencies

To support **clean dependency injection**, we update the environment so it **automatically provides dependencies** based on what each tool explicitly requires.

```python
class PythonEnvironment(Environment):
    def execute_action(self, agent, action_context: ActionContext,
                      action: Action, args: dict) -> dict:
        """Execute an action with automatic dependency injection."""
        try:
            # Create a copy of args to avoid modifying the original
            args_copy = args.copy()

            # If the function wants action_context, provide it
            if has_named_parameter(action.function, "action_context"):
                args_copy["action_context"] = action_context

            # Inject properties from action_context that match _prefixed parameters
            for key, value in action_context.properties.items():
                param_name = "_" + key
                if has_named_parameter(action.function, param_name):
                    args_copy[param_name] = value

            # Execute the function with injected dependencies
            result = action.execute(**args_copy)
            return self.format_result(result)
        except Exception as e:
            return {
                "tool_executed": False,
                "error": str(e)
            }
```

---

### ✅ What’s Happening Here

The environment is now **intelligent** about dependency management:

#### 🔹 Two Types of Injection:

1. **Special parameter names** like `action_context` are automatically injected
2. **Properties from the `action_context`** are injected when:

   * The tool function declares a parameter like `_auth_token`
   * The context has a value for `"auth_token"`

#### 🔹 Advantages:

* Tools **only receive** what they need
* Clean separation of concerns
* Promotes **security**, **modularity**, and **testability**
* Makes the **agent orchestration simple and clean**




Let's break this down line-by-line so you can see **exactly how the code achieves intelligent dependency management** and why this design pattern is powerful.

---

### 🔹 TWO TYPES OF INJECTION

#### 1. **Special Parameter: `action_context`**

```python
if has_named_parameter(action.function, "action_context"):
    args_copy["action_context"] = action_context
```

✅ **What this does:**
If the tool function **explicitly asks for `action_context`**, the environment injects the full context object.
This allows the tool to programmatically access *any part* of the environment (memory, LLM, auth, etc.) if needed.

---

#### 2. **\_Prefixed Parameters from ActionContext Properties**

```python
for key, value in action_context.properties.items():
    param_name = "_" + key
    if has_named_parameter(action.function, param_name):
        args_copy[param_name] = value
```

✅ **What this does:**
For every key in the context (e.g. `"auth_token"`, `"user_config"`):

* It checks if the function wants `_auth_token`, `_user_config`, etc.
* If yes, it injects the value directly into the function call.

🔍 **Why the underscore (`_`)?**
This is a naming convention to **signal that the value is being injected**, not passed by the user.

---

### 🔹 ADVANTAGES (and how the code supports them)

---

#### ✅ Tools only receive what they need

**How it's achieved:**

* The environment checks each tool's function signature using `has_named_parameter()`
* Only the matching parameters are injected
* If a tool doesn’t ask for something (e.g. `_auth_token`), it doesn’t receive it

---

#### ✅ Clean separation of concerns

**How it's achieved:**

* **Agent** decides *what* to do
* **Environment** decides *how* to do it
* **Tool** focuses only on its task

Each component has one job, and they don’t need to know how the others work internally.

---

#### ✅ Promotes security, modularity, and testability

* **Security:** Tools can’t access sensitive data (e.g. auth tokens) unless they declare they need them
* **Modularity:** Tools remain standalone and reusable; they don’t depend on the agent or other tools
* **Testability:** You can test tools by passing only their needed inputs (no need for full app context)

---

#### ✅ Agent logic remains simple and clean

**How it's achieved:**
The agent simply does:

```python
self.environment.execute_action(...)
```

…without worrying about what tools need or how to construct arguments.

It delegates the **messy orchestration** to the environment, keeping the agent clean and focused.

---



Let’s walk through a **small, concrete example** of how this intelligent dependency injection works using the `Environment`, `ActionContext`, and a couple of tools.

---

## 🛠️ Scenario: Greeting and Profile Update Agent

### 🔧 We have two tools:

1. `say_hello` – only needs a `name`
2. `update_profile` – needs a `username` and an `_auth_token` from the context

---

### ✅ 1. Define the tools

```python
from agentkit import register_tool

@register_tool(description="Say hello to a user")
def say_hello(name: str) -> str:
    return f"Hello, {name}!"

@register_tool(description="Update user profile with auth")
def update_profile(username: str, _auth_token: str) -> str:
    return f"Updated profile for {username} using token {_auth_token}"
```

---

### ✅ 2. Create an `ActionContext`

```python
from agentkit import ActionContext

# This holds injected dependencies
context = ActionContext({
    "auth_token": "abc123"
})
```

---

### ✅ 3. Tool registry and agent setup

```python
tool_registry = {
    "say_hello": say_hello,
    "update_profile": update_profile,
}
```

---

### ✅ 4. Environment with smart injection logic

```python
class Environment:
    def __init__(self, registry, context):
        self.registry = registry
        self.context = context

    def execute_tool(self, tool_name, args):
        tool = self.registry[tool_name]
        args_copy = args.copy()

        # Inject action_context if needed
        if "action_context" in tool.__code__.co_varnames:
            args_copy["action_context"] = self.context

        # Inject any _prefix dependencies from context
        for key, value in self.context.properties.items():
            param_name = "_" + key
            if param_name in tool.__code__.co_varnames:
                args_copy[param_name] = value

        return tool(**args_copy)
```

---

### ✅ 5. Simulate agent behavior

```python
env = Environment(tool_registry, context)

# Agent says hello
result1 = env.execute_tool("say_hello", {"name": "Alice"})
print(result1)  # "Hello, Alice!"

# Agent updates profile (with injected _auth_token)
result2 = env.execute_tool("update_profile", {"username": "Alice"})
print(result2)  # "Updated profile for Alice using token abc123"
```

---

## 🧠 Summary of What Just Happened

| Tool             | Needs Auth? | Received `auth_token`? | Why?                                  |
| ---------------- | ----------- | ---------------------- | ------------------------------------- |
| `say_hello`      | ❌ No        | ❌ No                   | It didn’t declare `_auth_token` param |
| `update_profile` | ✅ Yes       | ✅ Yes                  | It declared `_auth_token` param       |

---

This shows exactly how:

* The **environment** inspects the function’s needs
* **Injects** only what’s required
* Keeps the **agent** free from orchestration complexity
